# Predicting flight delays

*W261-004, Fall 2020*

*Team 21: Brian A., Karen W., Michael Z.* 

**Selected baselines and state-of-the-art literature for comparison**

| Author (pub. yr)   |  Data source     | No. of samples  | No. of model features  | Approach to data imbalance  | Map/reduce? | Model             | Accuracy | Recall | Precision | F1-score |
|--------------------|------------------|-----------------|------------------------|-----------------------------|-------------|-------------------|----------|--------|-----------|----------|
| Belcastro (2014)   | AOTP, QCLCD      | 31 mil, 233 mil | Up to ~246             | Under-sampling              | Yes         | Random Forest     | 0.74     | none   | none      | none     |
| Chakrabarty (2019) | USD BTS          | 97,360          | 12                     | SMOTE                       | No          | GB Classifier     | 0.86     | 0.86   | 0.88      | 0.85     |
| Etani (2019)       | FLIGHTSTATS, JMA | ~3000           | 23                     | Class weight parameter      | No          | SVM               | 0.70     | 0.70   | 0.84      | 0.76     |
|                    |                  |                 |                        |                             |             | Gradient Boosting | 0.76     | 0.91   | 0.78      | 0.84     |
|                    |                  |                 |                        |                             |             | Random Forest     | 0.71     | 0.81   | 0.78      | 0.79     |
|                    |                  |                 |                        |                             |             | AdaBoost          | 0.73     | 0.92   | 0.74      | 0.82     |
|                    |                  |                 |                        |                             |             | Decision Tree     | 0.74     | 0.89   | 0.77      | 0.82     |
| Patgiri (2020)     | ASA, NWS NDFD    | 1,566,226       | 8                      | none                        | Yes         | Logistic Regress. | 0.62     | 0.67   | 0.83      | 0.74     |
|                    |                  |                 |                        |                             |             | KNN               | 0.78     | 0.94   | 0.81      | 0.87     |
|                    |                  |                 |                        |                             |             | Gaussian NB       | 0.78     | 0.93   | 0.82      | 0.87     |
|                    |                  |                 |                        |                             |             | Decision Tree Cl. | 0.77     | 0.92   | 0.81      | 0.87     |
|                    |                  |                 |                        |                             |             | Random Forest     | 0.82     | 0.99   | 0.82      | 0.90     |

- SVM: Support Vector Machine
- KNN: K-Nearest Neighbor
- NB: Naive Bayes
- SMOTE: Synthetic Minority Over-Sampling Technique
- AOTP: Airline On-Time Performance
- QCLCD: Quality Controlled Local Climatological Data
- USD BTS: US Department's Bureau of Transportation Statistics 
- JMA: Japan Meteorological Agency
- ASA: American Statistical Association
- NWS NDFD: National Weather Service's National Digital Forecast Database

Some recent journal articles that are relevant to predicting flight delays are shown in the table above. They use a variety of data sources, three of which include flight and weather data (Belcastro, Etani, Patgiri), while one just uses flight data (Chakrabarty). One study (Belcastro) uses 31 million flights with 233 million weather reports filtered down to the relevant airport stations, while another one (Etani) only uses one month of data, or about 3000 flights. The number of features modeled in these studies ranges from 8 to about 246. The largest number of features (Belcastro) is due to 12 hours of weather at both the origin and the destination airports; these include data that could not be known prior to departure, such as the destination weather at the scheduled arrival time. Since delayed flights do not occur as often as on-time flights, the authors dealt with the class imbalance in different ways. Belcastro used under-sampling of the on-time data, Chakrabarty used over-sampling of the delayed flights, Etani used weight parameters, and Patagini did not mention any technique. The two authors who modeled the most data also used the map/reduce which is relevant to our project. The authors used a variety of models, including: random forest, gradient boosting, support vector machines, decision trees, K-nearest neighbor, and logistic regression. Belcastro and Chakrabarty concentrated on using just one model for their prediction, whereas the other two authors used five each.  The best model accuracy was 0.86 and came from Chakrabarty who used about 100,000 flights, 12 features, and oversampling of the delay data, while the lowest accuracy was 0.62 with logistic regression (Patgiri). Overall, random forest was used by three of the four authors with varying degrees of effectiveness.

In [0]:
# READ IN DATA SOURCES

airlines = spark.read.option("header", "true").parquet(f"dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data/201*.parquet")
weather = spark.read.option("header", "true").parquet(f"dbfs:/mnt/mids-w261/datasets_final_project/weather_data/*.parquet")

## 2. EDA and discussion of challenges

We focused on the following areas for data exploration:
* Bivariate analysis of feature vs outcome
* Correlation between features
* Data quality, including missingness, outliers
* Class balance of outcome

We limited our exploration of features to those that would be usable in predicting the outcome ≥2 hours prior to departure.

### 2.1 Bivariate analysis

Bivariate analysis of feature vs. outcome was important to determine which features were likely to influence departure delay. Examples of features that showed more variability with respect to outcome were departure delay for the previous leg of the flight, mean delay for flights at the airport for the previous day, and average wind speed.

In [0]:
# READ DATA AND PLOT BIVARIATE ANALYSES
aw = spark.read.parquet("/Users/michael.zhu@ischool.berkeley.edu/final/aw.parquet").cache()

# convert fl_date to int
datetonum = f.udf(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d").timetuple().tm_yday)
aw = aw.withColumn("DAY_OF_YEAR", datetonum(f.col("FL_DATE")).cast(IntegerType())) \
.drop("FL_DATE")

# Set Y axis limits
yMaxNoDelay = axesH[20, 0].get_ylim()[1]
yMaxDelay = axesH[20, 1].get_ylim()[1]
[0, np.max([yMaxNoDelay, yMaxDelay])]

defaultBins = 20
numBinsPerFeature = {'YEAR': 5, 'MONTH': 12, 'DAY_OF_MONTH': 30, 'DAY_OF_YEAR': 365, 'DAY_OF_WEEK': 7, 'HOUR': 24
                     , 'ORIGIN': defaultBins, 'DEST': defaultBins
                     , 'DEP_DELAY_NEW_p': defaultBins, 'CARRIER_DELAY_p': defaultBins, 'WEATHER_DELAY_p': defaultBins
                     , 'NAS_DELAY_p': defaultBins, 'SECURITY_DELAY_p': defaultBins, 'LATE_AIRCRAFT_DELAY_p': defaultBins 
                     , 'totNumFlights': defaultBins, 'meanDelayAcrossFlights': defaultBins, 'propDelayed': defaultBins
                     , 'avgTMP_airTemp': defaultBins, 'avgWND_speed': defaultBins, 'avgCIG_height': defaultBins, 'avgVIS_distance': defaultBins
                     , 'avgSLP_airPressure': defaultBins, 'avgPRECIP_duration': defaultBins, 'avgPRECIP_amount': defaultBins, 'avgSNOW_depth': defaultBins
                    }

# Create delayed/not delayed dataframes
notDelayedDf = aw.filter(f.col('DEP_DEL15')==0).cache()
delayedDf = aw.filter(f.col('DEP_DEL15')==1).cache()

nSubRows = len(numBinsPerFeature.keys())
nSubCols = 2
subplotCounter = 0

# Plot distributions of continuous variables by true label
figH, axesH = plt.subplots(nSubRows, nSubCols, tight_layout=True, figsize=(15,5*nSubRows))

for featureName in numBinsPerFeature:
  
  numBins = numBinsPerFeature[featureName]
  
  (featureDataNotDelayed, featureDataDelayed) = (notDelayedDf.select(featureName).dropna().rdd.flatMap(lambda element: element).collect()
                                                 , delayedDf.select(featureName).dropna().rdd.flatMap(lambda element: element).collect()
                                                )

  # plot
  for classInd in np.arange(2):

    if classInd == 0:
      if featureName in ['ORIGIN', 'DEST']:
        numBins = notDelayedDf.select(featureName).dropna().distinct().count()
      
      axesH[subplotCounter, classInd].hist(featureDataNotDelayed, bins=numBins, density=True)
      axesH[subplotCounter, classInd].set_title(featureName+'_NOTdelayed')
    
    if classInd == 1:
      if featureName in ['ORIGIN', 'DEST']:
        numBins = delayedDf.select(featureName).dropna().distinct().count()
              
      axesH[subplotCounter, classInd].hist(featureDataDelayed, bins=numBins, density=True, color='orange')
      axesH[subplotCounter, classInd].set_title(featureName+'_delayed')
          
    # cut off y axis for distributions with large densities
    if featureName in ['DEP_DELAY_NEW_p', 'CARRIER_DELAY_p', 'WEATHER_DELAY_p', 'NAS_DELAY_p', 'SECURITY_DELAY_p', 'LATE_AIRCRAFT_DELAY_p', 'meanDelayAcrossFlights']:
      yLims = [0, 0.00005]
    elif featureName == 'avgVIS_distance':
      yLims = [0, 0.00004]
    elif featureName == 'avgPRECIP_duration':
      yLims = [0, 0.075]
    elif featureName == 'avgPRECIP_amount':
      yLims = [0, 0.0001]
    elif featureName == 'avgSNOW_depth':
      yLims = [0, 0.005]
    else:
      yLims = axesH[subplotCounter, classInd].get_ylim()
      
    axesH[subplotCounter, classInd].set_ylim(yLims)
  
  subplotCounter += 1

d <img src="https://github.com/KarenKailun/img/raw/main/2.png", width=800>

Most other features showed little variability with respect to outcome. While these features did not show any obvious association with outcome, we erred on the side of including them because they could have important interactions with other features. However, knowing that these features showed little variability with respect to outcome allows us to deprioritize them if we encounter scalability issues.

### 2.2 Correlations between features
Correlations between features allow us to identify collinear features and eliminate those that may be redundant. We removed certain features that were likely to be redundant a priori; for example, `QUARTER` is redundant with `MONTH`, and air temperature is collinear with dewpoint temperature. We examined collinearity for continuous variables using scatterplots. (Note: some of the features in the correlation matrix below are derived in Section 3 Feature Engineering.)

In [0]:
# Correlation matrix plot

from pyspark.mllib.stat import Statistics
import seaborn as sns

#features = ['WND DIR','WND SP','CIG','VIS','TMP','DEW','SLP']
features = df2.columns[0:5] + df2.columns[6:15] + df2.columns[16:26]

corr = Statistics.corr(featuresRDD,  method="pearson")
fig, ax = plt.subplots(figsize=(11, 9))
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
cmap = sns.diverging_palette(240, 10, as_cmap=True)
ticks = features
sns.heatmap(corr, mask=mask, cmap=cmap, center=0, linewidths=.5, yticklabels=ticks,xticklabels=ticks, vmin=-1, vmax=1)
plt.title("Correlations between features.")
plt.xticks = features
display(plt.show())

<img src="https://github.com/KarenKailun/img/raw/main/image.png", width=800>

`MONTH` and `DAY_OF_YEAR` are strongly correlated; month could be dropped if needed. Several of the variables related to delay of the prior leg of the flight, such as the specific types of delay (e.g., `CARRIER_DELAY_p`) correlate with the summary delay variable, `DEP_DELAY_NEW_p`. The mean delay across all flights from that airport for the previous day correlates with the proportion of all flights delayed from that airport for the previous day. Certain weather features have some positive or negative collinearity, such as ceiling height and visibility distance. Of the variables that correlate with the target `DEP_DEL15`, the strongest ones relate to delay in the previous leg of the flight, and delay metrics for the previous day from that airport. Of note, none of the weather features correlate very strongly with delay of the previous flight leg, which suggests that the weather at the destination airport is less likely to improve model performance.

### 2.3 Data quality
Understanding the **data quality** is key for feature selection and decisions about imputation of values. We dropped a small number of records for which the outcome was null — these flights were canceled without incurring delay first. The weather data had a large number of features with values marked as missing or with poor quality codes. We dropped several weather features for which a large proportion were missing (e.g., EXAMPLE). We also a priori dropped weather observations where the temperature data were missing; we chose temperature as a sentinel feature for data quality, assuming that temperature is one of the simpler observations to measure and was less often missing. This was a simple rule to implement, reduced our data size, and eliminated not only observations missing temperature data, but often poor quality weather records in general. For the weather data, understanding how null values were recoded as numeric (e.g., 9999) was necessary for preprocessing features. Some weather data were noted to have truncated values (e.g., for ceiling, an arbitrary upper limit was used to mean clear skies).

### 2.4 Class imbalance

Understanding the class balance in outcome is critical to implementing measures to counteract potential class imbalance. Flights with departure delays of >15 minutes comprise only ~18% of all flights. 

| Year | Flights    | On-time, % | Delayed, % | Other, % |
|------|------------|------------|------------|----------|
| 2015 | 5,819,079  | 80.3       | 18.2       | 1.5      |
| 2016 | 5,617,658  | 81.9       | 17.0       | 1.1      |
| 2017 | 5,674,621  | 80.7       | 17.9       | 1.4      |
| 2018 | 7,213,446  | 80.3       | 18.1       | 1.6      |
| 2019 | 7,422,037  | 79.9       | 18.4       | 1.8      |

### *Implications of EDA for algorithm selection*

Our EDA has the following implications for predictive modeling:
* Several features are missing. Even after selectively dropping certain features, there is likely to be some missingness in the data. We will need to think about how to handle missing data for models that require full specification of the feature space, such as tree-based methods. 
* There are few features that correlate with the binary target variable, and several of the most strongly correlated are categorical. This means that logistic regression and tree-based models are likely to be reasonable choices for initial modeling efforts. Feature selection through regularization or selecting a threshold feature importance may be helpful, as even among the curated features, there are still a handful of features that appear to matter more than the others. Most of the weather features do not appear to correlate with outcome, and these may be eliminated iteratively in feature selection. 
* There are correlations between certain features of interest, and their interactions may be important in predicting outcome. Tree-based methods, which model these interactions implicitly, may be more flexible than logistic regression models where interactions would need to be specified explicitly.
* Certain categorical features, such as origin airport, have several discrete values. If one-hot encoding is required, such as for logistic regression, this will greatly expand the feature space and increase sparsity. Tree-based methods that can handle high cardinality categorical features may keep the feature space more compact, but the maximum number of bins for the trees will also need to be larger than the maximum number of classes. 
* The severe imbalance of the target outcome suggests that any algorithm chosen is likely to improve with strategies to handle class imbalance.

## 3. Feature Engineering

### 3.1 Explanation of native features

#### *Flight data*
Five years of flight data, from 2015 through 2019, were sourced from the US Department of Transportation's Bureau of Transportation Statistics. There are 109 features falling under the following categories: Time Period, Airline, Origin, Destination, Departure Performance, Arrival Performance, Cancellations and Diversions, Flight Summaries, Cause of Delay, Gate Return Info at Origin, and Diverted Airport Info. The features of interest are the date and time of the flight, the origin and destination airports, the airline and flight tail number, and whether the flight had a minimum 15 minute delay. Other features of interest for exploratory data analysis are the different causes for delays, as defined by the Bureau of Transportation Statistics:

* Air Carrier: The cause of the cancellation or delay was due to circumstances within the airline's control (e.g. maintenance or crew problems, aircraft cleaning, baggage loading, fueling, etc.).
* Extreme Weather: Significant meteorological conditions (actual or forecasted) that, in the judgment of the carrier, delays or prevents the operation of a flight such as tornado, blizzard or hurricane.
* National Aviation System (NAS): Delays and cancellations attributable to the national aviation system that refer to a broad set of conditions, such as non-extreme weather conditions, airport operations, heavy traffic volume, and air traffic control.
* Late-arriving aircraft: A previous flight with the same aircraft arrived late, causing the present flight to depart late.
* Security: Delays or cancellations caused by evacuation of a terminal or concourse, re-boarding of aircraft because of security breach, inoperative screening equipment and/or long lines in excess of 29 minutes at screening areas. 

**Cause of Delayed Flights**

|   Year   | Late-arr. plane,% | Air carrier,% |  NAS, %  | Extr. weather,% | Security, % |
|----------|-------------------|---------------|----------|-----------------|-------------|
|   2015   |       44.5        |     34.5      |   16.1   |      4.7        |     0.2     |
|   2016   |       44.7        |     33.6      |   17.4   |      4.1        |     0.2     |
|   2017   |       46.0        |     31.4      |   18.7   |      3.8        |     0.2     |
|   2018   |       45.2        |     31.6      |   18.2   |      4.8        |     0.2     |
|   2019   |       45.4        |     31.7      |   18.2   |      4.5        |     0.2     |

#### *Weather data*
Weather data, from 2015 through 2019 and from stations all across the US, were from the National Oceanic and Atmospheric Administration's repository at the National Climatic Data Center. Each weather report includes information about the station, as well as, the weather data. Important station data include a unique station identifier, the date and time of the report, and the station location defined by latitude and longitude. The weather data include about 170 features, but the main features with the most data are wind direction and speed, vertical visibility, horizontal visibility, air and dew temperatures, and the air pressure relative to mean sea level (see table below). These features have defined maximum and minimum values that could be used to check new data to be sure that they were within the expected range. Some have a scaling factor of 10 such as the air and dew temperatures. Units are in meters, meters per second, degrees Celsius, or hectopascals, depending on the feature. Missing data are represented as 999, 9999 or 99999. In addition, each data point has an associated quality code. Data points that have a quality code of 2, 3, 6 or 7 are considered suspect or erroneous and should not be used in modeling. Other features of interest, but with fewer entries, include liquid-precipitation and snow-depth. Most of the weather features do not have values listed, but a general classification of the current weather is represented as feature MW1 with a scale of 0 to 80 for indicating rain, drizzle, ice, fog, snow, and even blowing dust or sand.

#### *Station location data with respect to airports*
One obstacle in this project was to link the weather data from the closest station to each airport and the flight data. Weather reports included the latitude and longitude of the station, so the task was to find the latitude and longitude of each airport since it was not listed in the flight data. This <a href="https://www.aviationweather.gov/docs/metar/stations.txt">dataset from aviationweather.gov</a> lists weather stations, their latitude and longitude, and the airport where they are located, if applicable.

### Selected weather features 

| Code      |      Units      |   Min   |   Max   | Scaling factor | Brief description                  |  
|-----------|-----------------|---------|---------|----------------|------------------------------------|
| WND       | angular degrees |       1 |     360 |         1      | Direction wind is blowing          |
| WND       | meters/sec      |       0 |     900 |        10      | Horizontal wind speed              |
| CIG       | meters          |       0 |   22000 |         1      | Vertical visibility                |
| VIS       | meters          |       0 |  160000 |         1      | Horizontal visibility              |
| TMP       | degrees Celsius |    -932 |     618 |        10      | Air temperature                    |
| DEW       | degrees Celsius |    -982 |     368 |        10      | Temperaure for saturation to occur |
| SLP       | hectopascals    |    8600 |   10900 |        10      | Air pressure relative to sea level |
| AA2       | hours           |       0 |      98 |         1      | Duration for rain data collection  |
| AA2       | mm              |       0 |    9998 |        10      | Depth of liquid precipitation      |
| AJ1       | cm              |       0 |    1200 |         1      | Depth of snow and ice on ground    |


Federal Climate Complex Data Documentation for Integrated Surface Data (ASD), January 12, 2018;
https://www.ncei.noaa.gov/data/global-hourly/doc/isd-format-document.pdf

### 3.2 Treatment of missing values

For the outcome variable, observations with missing values were dropped. For features, missing values were either dropped or imputed according to the feature. 

* Flights where the tail number was missing were dropped; these were relatively few observations and allowed us to keep a full dataset with respect to the previous flight leg’s information. 
* Weather observations where the temperature data were missing were dropped. We chose temperature as a sentinel feature for data quality, assuming that temperature is one of the simpler observations to measure and was less often missing. This was a simple rule to implement, reduced our data size, and eliminated not only observations missing temperature data, but also poor quality weather records in general. 
* Observations with missing data on wind speed, precipitation duration and amount, ceiling height, and visibility were imputed with the mean of these variables. While this adds some noise to our model, it allows us to keep more data for the random forest algorithm. 
* Observations missing data on snow depth were imputed with zero, assuming that these data are often missing when there is no snow.

### 3.3 Importance of algorithm choice to justify feature transformation choices

We spent significant time on feature engineering, because the literature suggests that a fewer number of informative features will predict better than a large number of low-quality features. 

For a baseline model, we used logistic regression. Our final model used random forest. These algorithms required different feature engineering choices. While for the baseline logistic regression model, we performed one-hot encoding, scaling, and cyclical transformation, we chose not to perform these transformations for the random forest algorithm for the following reasons: 
* One-hot encoding introduces sparsity into the data which is undesirable for the random forest algorithm. Instead, we ensured that true categorical variables were not coded as ordinal variables. 
* Normalization according to the training data was helpful for our baseline logistic regression model given the differences in scale and the skewness of certain features, but is not needed for the random forest algorithm where the decision trees will partition the data based on the split of the data that maximizes purity. 
* A transformation that captures the cyclical nature of certain variables, such as day of the year, is theoretically desirable. However, performing sine/cosine transformations, while useful for logistic regression, are problematic for tree-based models because each feature would be evaluated individually, and in a random forest, not every feature is evaluated. Therefore, splitting a cyclical variable into sine and cosine components could result in only one component being included in a tree. We performed the following:
  * For cyclical variables with few discrete values (e.g., day of the week), we treated these as categorical. 
  * For those with many possible values (e.g., day of the year), we left them as ordinal variables and acknowledge that we are trading a more accurate representation of the cyclical feature for the benefits of a smaller feature space. 
* Interaction terms were not explicitly modeled for the random forest model, as these are handled implicitly by the algorithm.

### 3.4 Joins

An overview of joins is shown below:

<img src="https://github.com/KarenKailun/img/raw/main/joins.png", width=800>

#### *Joining previous flight leg*

We joined flight data with information about the preceding leg of the flight — this is an intuitive choice but also supported by models of delay propagation. To do this efficiently, we ordered the flight data by date and time, horizontally concatenated it to itself shifted by one row, then filtered out data where the tail numbers did not match. For the previous flight, we examined the origin airport, minutes of departure delay, and contributing departure delay by type.

<img src="https://github.com/KarenKailun/img/raw/main/prev_flight_join.png", width=800>

In [0]:
# Joining flights to previous leg of flight data

# Define window --- whole df, ordered
w2  = Window.orderBy("FL_DATE", "TAIL_NUM", "CRS_DEP_DATETIME_UTC")

# row index d1
d1 = air_f.select("FL_DATE", "TAIL_NUM", "CRS_DEP_DATETIME_UTC") \
.withColumn("row_number",f.row_number().over(w2)-1)

# Get delay info from previous flight (d2)
d2 = air_f.select("FL_DATE", "TAIL_NUM", "CRS_DEP_DATETIME_UTC", f.col("ORIGIN").alias("ORIGIN_p"), f.col("DEP_DELAY_NEW").alias("DEP_DELAY_NEW_p"), 
            f.col("CARRIER_DELAY").alias("CARRIER_DELAY_p"), f.col("WEATHER_DELAY").alias("WEATHER_DELAY_p"), 
            f.col("NAS_DELAY").alias("NAS_DELAY_p"), f.col("SECURITY_DELAY").alias("SECURITY_DELAY_p"), f.col("LATE_AIRCRAFT_DELAY").alias("LATE_AIRCRAFT_DELAY_p")) \
.withColumn("row_number",f.row_number().over(w2)) \
.withColumnRenamed("TAIL_NUM", "TAIL_NUM_p") \
.withColumnRenamed("FL_DATE", "FL_DATE_p") \
.drop("CRS_DEP_DATETIME_UTC")

# Join flights
d3 = d1.join(d2, ["row_number"],'inner') \
.drop("row_number") \
.filter((f.col("FL_DATE")==f.col("FL_DATE_p")) & (f.col("TAIL_NUM")==f.col("TAIL_NUM_p"))) \
.drop("TAIL_NUM_p", "FL_DATE_p").cache()

#### *Joining autocorrelations of airport activity and delays from previous day*

During data exploration, we noted autocorrelation of measures of airport activity and delay (e.g., proportion of flights delayed per day) over time.

<img src="https://github.com/KarenKailun/img/raw/main/propdelay.png", width=600>

These may reflect periods of high travel, such as holidays. They can also reflect delay propagation over several days, or multi-day events that affect a certain airport (e.g., severe inclement weather). We engineered additional features to reflect airport activity and proportion of delays for the previous day:
* Number of flights on the previous day at that airport (`totNumFlights`)
* Average delay in minutes for all flights from the previous day at that airport (`meanDelayAcrossFlights`)
* Proportion of flights delayed from the previous day at that airport (`propDelayed`)

These metrics were aggregated efficiently using map-reduce, then joined to the flight data on a 1-day lag.

In [0]:
# Join flight data to airport metrics from previous day

def mapper_convertToDict(reducedLine):
  date, airportStr = reducedLine[0]
  allDelayTimesStrs = reducedLine[1]
  allDelayTimes = [float(delay) for delay in allDelayTimesStrs]
  
  totNumFlights = len(allDelayTimesStrs)
  meanDelay = float(np.mean(allDelayTimes))
  propDelayed = float(sum(np.array(allDelayTimes) >= 15)/totNumFlights)
  
  airportDict = {airportStr: {'totNumFlights': totNumFlights, 'meanDelayMins': meanDelay, 'propDelayed': propDelayed}}
  
  return (date, airportDict)

def flatmap_dateAndAirport(line):
  flightDate, allAirportsDict = line
  
  for airport in list(allAirportsDict.keys()):
    yield (flightDate, airport, allAirportsDict[airport]['totNumFlights'], allAirportsDict[airport]['meanDelayMins'], allAirportsDict[airport]['propDelayed'])

# create a separate df with only delay data to work with
airlinesDelayedData = air_prev.select(['ORIGIN', 'ORIGIN_AIRPORT_ID', 'DEP_DELAY', 'FL_DATE', 'DEP_DEL15'])

delayMetricsDf = airlinesDelayedData.dropna().rdd.map(lambda line: [column for column in line]) \
                                                   .map(lambda line: ((datetime.datetime.strptime(line[2], '%Y-%m-%d'), line[0]), (str(line[3]),))) \
                                                   .reduceByKey(lambda currLine, nextLine: currLine + nextLine) \
                                                   .map(lambda line: mapper_convertToDict(line)) \
                                                   .flatMap(lambda line: flatmap_dateAndAirport(line)) \
                                                   .toDF(('flightDate', 'airportCode', 'totNumFlights', 'meanDelayAcrossFlights', 'propDelayed'))

def get_dayBeforeFlight(flightDateStr):
  flightDatetime = datetime.datetime.strptime(flightDateStr, '%Y-%m-%d')
  dayBeforeFlight = flightDatetime - datetime.timedelta(days=1)
  
  return dayBeforeFlight

udf_get_dayBeforeFlight = f.udf(get_dayBeforeFlight, TimestampType())

air_prev = air_prev.withColumn('dayBeforeFlightDate_forJoin', udf_get_dayBeforeFlight('FL_DATE'))
                      
joinConds = [(air_prev.dayBeforeFlightDate_forJoin == delayMetricsDf.flightDate)
             , air_prev.ORIGIN == delayMetricsDf.airportCode]

airdf = air_prev.join(delayMetricsDf, joinConds).cache()

In [0]:
# Select specific features from airline data

air = airdf.select('DEP_DEL15', 'YEAR', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'CRS_DEP_TIME', 'OP_CARRIER', 'ORIGIN', 'DEST', 'CRS_DEP_DATETIME_UTC', 'ORIGIN_p', 'DEP_DELAY_NEW_p', 'CARRIER_DELAY_p', 'WEATHER_DELAY_p', 'NAS_DELAY_p', 'SECURITY_DELAY_p', 'LATE_AIRCRAFT_DELAY_p', 'FL_DATE', 'totNumFlights', 'meanDelayAcrossFlights', 'propDelayed') \
.where(airdf.DEP_DEL15.isNotNull()) \
.withColumn("DEP_DEL15", airdf["DEP_DEL15"].cast(FloatType())) \
.withColumn("YEAR", airdf["YEAR"].cast(IntegerType())) \
.withColumn("MONTH", airdf["MONTH"].cast(IntegerType())) \
.withColumn("DAY_OF_MONTH", airdf["DAY_OF_MONTH"].cast(IntegerType())) \
.withColumn("DEP_DELAY_NEW_p", airdf["DEP_DELAY_NEW_p"].cast(FloatType())) \
.withColumn("CARRIER_DELAY_p", airdf["CARRIER_DELAY_p"].cast(FloatType())) \
.withColumn("WEATHER_DELAY_p", airdf["WEATHER_DELAY_p"].cast(FloatType())) \
.withColumn("NAS_DELAY_p", airdf["NAS_DELAY_p"].cast(FloatType())) \
.withColumn("SECURITY_DELAY_p", airdf["SECURITY_DELAY_p"].cast(FloatType())) \
.withColumn("LATE_AIRCRAFT_DELAY_p", airdf["LATE_AIRCRAFT_DELAY_p"].cast(FloatType())) \
.fillna({"CARRIER_DELAY_p":0, "WEATHER_DELAY_p":0, "NAS_DELAY_p":0, "SECURITY_DELAY_p":0, "LATE_AIRCRAFT_DELAY_p":0}) \
.cache()

# dbutils.fs.rm("/Users/michael.zhu@ischool.berkeley.edu/final/air.parquet", True)
# air.write.parquet("/Users/michael.zhu@ischool.berkeley.edu/final/air.parquet")
# air = spark.read.parquet("/Users/michael.zhu@ischool.berkeley.edu/final/air.parquet")

#### *Joining weather to flight data*

Weather-related delays comprise a relatively small proportion of delays overall. To join the weather data, we used another dataset of weather stations and IATA codes (https://www.aviationweather.gov/docs/metar/stations.txt). Because of the size of the weather data, we filtered aggressively and reduced the feature space from the beginning based on the findings from EDA, which indicated that many of the features had low data completeness. 

Using a 1% sample of the full weather data, we identified distinct origin airports, which we then **inner-joined** to the weather stations data on IATA code to filter to only weather data collected at the origin airport. This was a **broadcast join**. We also selected only features of wind speed, ceiling height, visibility distance, air temperature, sea level pressure, precipitation duration and amount, and snow depth. We reduced the data further by aggregating weather measurements by the airport and hour --- this allowed us to perform a **many-to-one join**, rather than a many-to-many join. After imputing missing values in the weather data (see above, `Treatment of missing values`), we joined flights to weather data by flooring the flight departure time and performing an **equi join** to the weather data.

In [0]:
# FILTER WEATHER DATA TO STATIONS AT AIRPORTS

# Read data
us_stations = spark.sql("select * from us_stations_all")
weather = spark.read.option("header", "true")\
                    .parquet(f"dbfs:/mnt/mids-w261/datasets_final_project/weather_data/*.parquet")

# Get distinct airports from flights data, which we will use to filter weather data
airports = airdf0 \
.sample(False, .01) \
.select("ORIGIN") \a
.distinct().cache()

# Broadcast join the list of airports to the table of weather stations, to filter them to those at airports
stations_sm = airports.join(us_stations, airports.ORIGIN==us_stations.IATA, 'inner') \
.withColumn('ICAO', strip_udf(us_stations.ICAO)) \
.cache()

# select relevant features from weather data
w2 = weather.select("STATION", "CALL_SIGN", "DATE", 'WND', 'CIG', 'VIS', 'TMP', 'SLP', 'AA2', 'AJ1') \
.withColumn('CALL_SIGN', strip_udf(weather.CALL_SIGN)) 

# Inner join the airport stations data to the weather data to filter it, and parse the text fields
dropcols = ['WND', 'CIG', 'VIS', 'TMP', 'SLP', 'AA2', 'AJ1']
w3 = stations_sm \
.select("ORIGIN", "ICAO") \
.join(w2, stations_sm.ICAO==w2.CALL_SIGN, 'inner') \
.withColumn('WND_angle', f.split(f.col('WND'), ',').getItem(0).cast(FloatType())) \
.withColumn('WND_angleQuality', f.split(f.col('WND'), ',').getItem(1).cast(IntegerType())) \
.withColumn('WND_type', f.split(f.col('WND'), ',').getItem(2).cast(StringType())) \
.withColumn('WND_speed', f.split(f.col('WND'), ',').getItem(3).cast(FloatType())) \
.withColumn('WND_speedQuality', f.split(f.col('WND'), ',').getItem(4).cast(IntegerType())) \
.withColumn('CIG_height', f.split(f.col('CIG'), ',').getItem(0).cast(FloatType())) \
.withColumn('CIG_heightQuality', f.split(f.col('CIG'), ',').getItem(1).cast(IntegerType())) \
.withColumn('CIG_type', f.split(f.col('CIG'), ',').getItem(2).cast(StringType())) \
.withColumn('CIG_CAVOK', f.split(f.col('CIG'), ',').getItem(3).cast(StringType())) \
.withColumn('VIS_distance', f.split(f.col('VIS'), ',').getItem(0).cast(FloatType())) \
.withColumn('VIS_quality', f.split(f.col('VIS'), ',').getItem(1).cast(IntegerType())) \
.withColumn('TMP_airTemp', f.split(f.col('TMP'), ',').getItem(0).cast(FloatType())) \
.withColumn('TMP_quality', f.split(f.col('TMP'), ',').getItem(1).cast(StringType())) \
.withColumn('SLP_airPressure', f.split(f.col('SLP'), ',').getItem(0).cast(FloatType())) \
.withColumn('SLP_quality', f.split(f.col('SLP'), ',').getItem(1).cast(IntegerType())) \
.withColumn('PRECIP_duration', f.split(f.col('AA2'), ',').getItem(0).cast(IntegerType())) \
.withColumn('PRECIP_amount', f.split(f.col('AA2'), ',').getItem(1).cast(FloatType())) \
.withColumn('PRECIP_conditionCode', f.split(f.col('AA2'), ',').getItem(2).cast(StringType())) \
.withColumn('PRECIP_quality', f.split(f.col('AA2'), ',').getItem(3).cast(StringType())) \
.withColumn('SNOW_depth', f.split(f.col('AJ1'), ',').getItem(0).cast(FloatType())) \
.withColumn('SNOW_conditionCode', f.split(f.col('AJ1'), ',').getItem(1).cast(StringType())) \
.withColumn('SNOW_quality', f.split(f.col('AJ1'), ',').getItem(2).cast(StringType())) \
.withColumn('SNOW_liquidDepth', f.split(f.col('AJ1'), ',').getItem(3).cast(FloatType())) \
.withColumn('SNOW_liquidCondition', f.split(f.col('AJ1'), ',').getItem(4).cast(StringType())) \
.withColumn('SNOW_liquidQuality', f.split(f.col('AJ1'), ',').getItem(5).cast(StringType())) \
.drop(*dropcols) \
.where(f.col("TMP_quality") != 9) \
.cache()

# save intermediate weather data
# w3.write.parquet("/Users/michael.zhu@ischool.berkeley.edu/final/w3.parquet")
w3 = spark.read.parquet("/Users/michael.zhu@ischool.berkeley.edu/final/w3.parquet")

In [0]:
# IMPUTE WEATHER DATA AND AGGREGATE TO THE HOUR BEFORE JOINING TO FLIGHTS

# Impute weather data based on specific null tokens
rep99_udf = udf(lambda x: None if x == 99 else x, FloatType())
rep9999_udf = udf(lambda x: None if x == 9999 else x, FloatType())
rep99999_udf = udf(lambda x: None if x == 99999 else x, FloatType())
rep999999_udf = udf(lambda x: None if x == 999999 else x, FloatType())

# UDFs to extract date and hour
def getdate(x):
  try:
    return x.date()
  except:
    return None
  
def gethour(x):
  try:
    return x.hour
  except:
    return None

getdate_udf = f.udf(getdate, DateType())
gethour_udf = f.udf(gethour, IntegerType())

# Replace 999s with nulls
wnull = w3 \
.withColumn("TMP_airTemp", rep9999_udf(w3.TMP_airTemp)) \
.withColumn("WND_speed", rep9999_udf(w3.WND_speed)) \
.withColumn("PRECIP_amount", rep9999_udf(w3.PRECIP_amount)) \
.withColumn("CIG_height", rep99999_udf(w3.CIG_height)) \
.withColumn("SLP_airPressure", rep99999_udf(w3.SLP_airPressure)) \
.withColumn("VIS_distance", rep999999_udf(w3.VIS_distance)) \
.withColumn("SNOW_depth", rep9999_udf(w3.SNOW_depth)) \
.withColumn("DATE_day", getdate_udf(w3.DATE)) \
.withColumn("HOUR", gethour_udf(w3.DATE)) \
.select("ORIGIN", "ICAO", "DATE", "DATE_day", "HOUR", "TMP_airTemp", "WND_speed", "CIG_height", "VIS_distance", "SLP_airPressure", "PRECIP_duration", "PRECIP_amount", "SNOW_depth") \
.cache()

# Aggregate weather observations to the hour
wnull_hour = wnull.groupBy("ORIGIN", "DATE_day", "HOUR").mean() \
.toDF(*(c.replace('avg(', 'avg').replace(')', '') for c in wnull_hour.columns)) \
.drop("avgHOUR") \
.cache()

# Get a reasonable mean for imputation
display(w3 \
        .withColumn("TMP_airTemp", rep9999_udf(w3.TMP_airTemp)) \
        .withColumn("WND_speed", rep9999_udf(w3.WND_speed)) \
        .withColumn("PRECIP_amount", rep9999_udf(w3.PRECIP_amount)) \
        .withColumn("CIG_height", rep99999_udf(w3.CIG_height)) \
        .withColumn("SLP_airPressure", rep99999_udf(w3.SLP_airPressure)) \
        .withColumn("VIS_distance", rep999999_udf(w3.VIS_distance)) \
        .select("TMP_airTemp", "WND_speed", "CIG_height", "VIS_distance", "SLP_airPressure", "PRECIP_duration", "PRECIP_amount", "SNOW_depth") \
        .describe())

# For quality control, drop at least those with poor temp reading
# Impute with mean or with zero (snow and precipitation duration)
weather_imp = wnull_hour \
.fillna({"avgTMP_airTemp":130.6, "avgWND_speed":36.0, "avgCIG_height":11571.6, "avgVIS_distance":13824.1, "avgSLP_airPressure": 10164.2, "avgPRECIP_duration":0, "avgPRECIP_amount":26.5, 'avgSNOW_depth':0}) \
.cache()

# save weather data
# dbutils.fs.rm("/Users/michael.zhu24@ischool.berkeley.edu/final/weather_imp.parquet", True)
# weather_imp.write.parquet("/Users/michael.zhu4@ischool.berkeley.edu/final/weather_imp.parquet")
weather_imp = spark.read.parquet("/Users/michael.zhu24@ischool.berkeley.edu/final/weather_imp.parquet")

In [0]:
# JOIN WEATHER TO FLIGHT DATA

# This is airline data with extra features
air = spark.read.parquet("/Users/michael.zhu24@ischool.berkeley.edu/final/air.parquet").cache()

# Reformat date/time features
air = air \
.withColumn("DATE_day", getdate_udf(air.CRS_DEP_DATETIME_UTC)) \
.withColumn("HOUR", gethour_udf(air.CRS_DEP_DATETIME_UTC)-2).cache()

# Define join conditions
joinConds = [(air.DATE_day == weather_imp.DATE_day)
             , air.HOUR==weather_imp.HOUR
             , air.ORIGIN==weather_imp.ORIGIN]

dropcols = [weather_imp.DATE_day, weather_imp.HOUR, weather_imp.ORIGIN]

# Inner join flight to weather data
aw = air \
.join(weather_imp, joinConds, 'inner') \
.drop(weather_imp.DATE_day) \
.drop(weather_imp.HOUR) \
.drop(weather_imp.ORIGIN) \
.cache()

### Model Features

After performing the joins and engineering features, we used the following features to train our models:

| Feature                |     Min |     Max | Brief description                                         |
|------------------------|---------|---------|-----------------------------------------------------------|
| YEAR                   |    2015 |    2019 | Year of flight departure                                  |
| MONTH                  |       1 |      12 | Month of flight departure                                 |
| DAY_OF_MONTH           |       1 |      31 | Day of month of flight departure                          |
| DAY_OF_YEAR            |       1 |     366 | Day of flight departure                                   |
| CRS_DEP_TIME           |       1 |    2359 | Flight time as reported by Computer Reservation System    |
| HOUR                   |       0 |      21 | Hour of flight departure                                  |
|                        |         |         |                                                           |
| avgTMP_airTemp         |    -439 |     494 | Average hourly air temperature (unit: degree Celsius/10)  |            
| avgWND_speed           |       0 |     412 | Average hourly wind speed (unit: (meter/sec)/10)          |
| avgCIG_height          |       0 |   22000 | Average hourly vertical visibility (unit: meter)          |
| avgVIS_distance        |       0 |  160000 | Average hourly horizontal visibility (unit: meter)        |
| avgSLP_airPressure     |    9601 |  105960 | Average hourly sea level pressure (unit: hectopascal/10)  |
| avgPRECIP_duration     |       0 |      24 | Average hourly time of precipitation collection (unit: hour)|
| avgPRECIP_amount       |       0 |    2421 | Average hourly amount of precipitation (unit: mm/10)      |
| avgSNOW_depth          |       0 |     107 | Average hourly amount of snow depth (unit: cm)            |
|                        |         |         |                                                           |
| totNumFlights          |       1 |    1228 | Total number of flights at airport on previous day        |
| meanDelayAcrossFlights |   -69.5 |    1970 | Mean time delay of flights at airport on previous day     |
| propDelayed            |       0 |       1 | Proportion of flights delayed at airport on previous day  |
|                        |         |         |                                                           |
| DEP_DELAY_NEW_p        |       0 |    2710 | Previous flight departure delay (unit: minute)            |
| CARRIER_DELAY_p        |       0 |    1843 | Previous flight carrier delay (unit: minute)              |
| WEATHER_DELAY_p        |       0 |    2692 | Previous flight weather delay (unit: minute)              |
| NAS_DELAY_P            |       0 |    1848 | Previous flight NAS delay (unit: minute)                  |
| SECURITY_DELAY_p       |       0 |     927 | Previous flight security delay (unit: minute)             |
| LATE_AIRCRAFT_DELAY_p  |       0 |    2454 | Previous flight late aircraft delay (unit: minute)        |
|                        |         |         |                                                           |
| DEP_DEL15              |       0 |       1 | Departure delay of 15 minutes or more (target variable)   |

## 4. Algorithm exploration

For our initial exploration of algorithms, we limited the data to flights from Atlanta or Chicago, and we used only 1 quarter (3 months) of data. 

**Logistic regression**
We selected regularized logistic regression as our baseline model. This algorithm was appropriate for a binary classification problem with a mix of numeric and categorical features that could be one-hot encoded. Use of logistic regression assumes a linear relationship between the logit of the outcome and each predictor variable. We previously identified high intercorrelations between predictors and removed redundant predictors. Our baseline model included the following:

 *Delay ~ dayofweek + year + month + dayofmonth + dayofyear + departuretime + OPcarrier + origin + destination + windspeed + ceilingheight + visibilitydistance + airtemperature + precipitationduration + precipitationamount + snowdepth*

For the baseline model, we used class weighting to adjust for imbalance in the outcome. We performed hyperparameter tuning using a grid search for the regularization parameter. Our best logistic regression model had an AUROC of approximately 0.7. 

This baseline model had several limitations: we included certain features with very low variance in the small sample data (e.g., `YEAR` had no variance). In the baseline model, missing weather data (`null`) were mostly zero-filled, and we had not adjusted the null encoding to include the special values (e.g., 9999). 

**Random forest**

The random forest classifier was chosen because of the strong performance of tree-based algorithms in the literature (see table in Section 1) and because the algorithm makes fewer assumptions about the mathematical relationship between predictors and the outcome. Random forest also does not necessarily require one-hot encoding, which keeps the feature space smaller; this offers advantages for scalability. We trained a random forest model using the same features and labels as in the baseline logistic regression. The initial AUROC was also approximately 0.7. 

We examined feature importance in this baseline random forest model. Temperature and visibility were found to be very important features; however, this baseline implementation did not include certain derived features, such as the previous flight leg, airport activity, and delay autocorrelation features.

<img src="https://github.com/KarenKailun/img/raw/main/baseRF.png", width=800>

## 5. Algorithm implementation

We selected the random forest classifier over logistic regression to refine our pipeline further, because it had relatively good performance in the baseline implementation, offered the possibility of a more compact feature space than logistic regression, and had strong support in the literature.

### 5.1 Toy example of random forest algorithm

Random forests is an ensemble method that uses multiple decision trees to output a prediction on input data. For predicting delayed flights, we used a random forest model for classification (0=no delay, 1=delay) based on flight and weather data gathered from 2015 to 2019. To demonstrate how this algorithm works, we will go through a toy example using the following small data set consisting of 8 data points (each one representing a different flight) to train the model:

| DEP_DEL15 | DEP_DELAY_NEW_p | propDelayed | meanDelayAcrossFlights |
|:---------:|:---------------:|:-----------:|:----------------------:|
|     1     |        42       |     0.25    |          13.6          |
|     0     |        0        |     0.95    |           0.7          |
|     0     |        17       |     0.20    |          14.1          |
|     0     |        0        |     0.16    |           8.4          |
|     0     |        0        |     0.07    |           1.7          |
|     0     |        42       |     0.17    |           8.2          |
|     0     |        0        |     0.12    |           1.9          |
|     1     |        0        |     0.14    |           6.0          |

We will try to predict the target variable “DEP_DEL15” using the 3 features represented in the other 3 columns. To start, we will build a single decision tree using the above data. 
A decision tree consists of multiple split points, each of which are determined by a threshold value of one of the features. The optimal feature and threshold value to use at each split point can be determined using Gini gain, which measures the difference in Gini impurity between a parent node and its child branches. For a given data set with \\(C\\) classes, the Gini impurity can be calculated with the equation: 

$$\sum_i^C p_i(1-p_i) = 1 - \sum_i^C p_i^2$$ 

This metric measures the probability of making an incorrect classification given all data points were randomly labeled according to the overall class distribution. Therefore, a perfect classifier would separate the data set into two parts, each with Gini impurity = 0. To calculate Gini gain in a decision tree, we simply subtract the weighted Gini impurity of the child branches from the Gini impurity of the parent node. Intuitively, Gini gain will give us a measure of how much “impurity” was removed as a result of making a split, therefore the algorithm aims to maximize this quantity at every split point. 

Using the 3 features in our toy data set, we can start building a decision tree by first calculating the initial Gini impurity before making any splits:

$$Gini = 1 - \sum_i^C p_i^2 = 1 - ((2/8)^2 + (6/8)^2) = 24/64 = \bold{3/8}$$   

Next, we can choose possible splits for the first node of the decision tree using our features at different thresholds (though in practice, the algorithm will do an exhaustive search of all possible splits, for brevity we will only choose 1 potential split for each feature). For example, if we were to make a split with DEP_DELAY_NEW_p > 18, we can calculate the Gini impurity of each branch that this split would create:

$$Left\ Branch = 1 - ((1/2)^2 + (1/2)^2) = 2/4$$

$$Right\ Branch = 1 - ((5/6)^2 + (1/6)^2) = 10/36$$ 

To calculate the weighted Gini impurity of these two branches, we can weight each value by the proportion of data points in each branch:

$$Gini\\_weighted = (2(2/4) + 6(10/36)) / 8 = 1/3$$

Now we can calculate the Gini gain of this potential split by subtracting the weighted Gini impurity from the initial Gini impurity of the entire data set:

$$Gini\ Gain = 3/8 - 1/3 \approx \bold{0.0416}$$

Next, we can repeat these steps to calculate the Gini gain of other potential splits using other features and thresholds. The calculations are summarized in the table below:

| Feature to try for split   | Left Branch Gini                | Right Branch Gini               | Weighted Gini                 | Gini Gain          |
|----------------------------|---------------------------------|---------------------------------|-------------------------------|--------------------|
| DEP_DELAY_NEW_p > 18       | 1 - ((1/2)^2 + (1/2)^2) = 2/4   | 1 - ((5/6)^2 + (1/6)^2) = 10/36 | (2(2/4) + 6(10/36)) / 8 = 1/3 | 3/8 - 1/3 = 0.0416 |
| propDelayed > 0.13         | 1 - ((4/6)^2 + (2/6)^2) = 16/36 | 1 - ((2/2)^2 + (0/2)^2) = 0     | (6(16/36) + 2(0)) / 8 = 1/3   | 3/8 - 1/3 = 0.0416 |
| **meanDelayAcrossFlights > 3** | **1 - ((3/5)^2 + (2/5)^2) = 12/25** | **1 - ((3/3)^2 + (0/3)^2) = 0**     | **(5(12/25) + 3(0)) / 8 = 3/10**  | **3/8 - 3/10 = 0.075 **|

Out of these 3 potential splits we choose the one with the highest Gini gain (bolded row), which uses the “meanDelayAcrossFlights” feature at a threshold value of 3. Splitting the data with this feature gives us the following tree:

<img src="https://github.com/KarenKailun/img/raw/main/toytree.png", width=400>

We see that one branch predicts the training data perfectly and the other branch has 5 data points left over. Therefore, we will continue splitting these 5 remaining data points, again using splits that maximize Gini gain. Repeating this process, we end up with the following calculations:

| Split Index | Feature to try for split    | Left Branch Gini                | Right Branch Gini               | Weighted Gini                 | Gini Gain            |
|-------------|-----------------------------|---------------------------------|---------------------------------|-------------------------------|----------------------|
|           1 | DEP_DELAY_NEW_p > 18        | 1 - ((1/2)^2 + (1/2)^2) = 2/4   | 1 - ((5/6)^2 + (1/6)^2) = 10/36 | (2(2/4) + 6(10/36)) / 8 = 1/3 | 3/8 - 1/3 = 0.0416   |
|             | propDelayed > 0.13          | 1 - ((4/6)^2 + (2/6)^2) = 16/36 | 1 - ((2/2)^2 + (0/2)^2) = 0     | (6(16/36) + 2(0)) / 8 = 1/3   | 3/8 - 1/3 = 0.0416   |
|             | **meanDelayAcrossFlights > 3**  | **1 - ((3/5)^2 + (2/5)^2) = 12/25** | **1 - ((3/3)^2 + (0/3)^2) = 0**     | **(5(12/25) + 3(0)) / 8 = 3/10**  | **3/8 - 3/10 = 0.075**   |
|           2 | DEP_DELAY_NEW_p > 18        | 1 - ((1/2)^2 + (1/2)^2) = 2/4   | 1 - ((2/3)^2 + (1/3)^2) = 4/9   | (2(2/4) + 3(4/9)) / 5 = 7/15  | 12/25 - 7/15 = 0.013 |
|             | **propDelayed > 0.15**          | **1 - ((3/4)^2 + (1/4)^2) = 6/16**  | **1 - ((0/1)^2 + (1/1)^2) = 0**     | **(4(6/16) + 1(0)) / 5 = 3/10**   | **12/25 - 3/10 = 0.18**  |
|             | meanDelayAcrossFlights > 7  | 1 - ((3/4)^2 + (1/4)^2) = 6/16  | 1 - ((0/1)^2 + (1/1)^2) = 0     | (4(6/16) + 1(0)) / 5 = 3/10   | 12/25 - 3/10 = 0.18  |
|           3 | DEP_DELAY_NEW_p > 18        | 1 - ((1/2)^2 + (1/2)^2) = 2/4   | 1 - ((2/2)^2 + (0/2)^2) = 0     | (2(2/4) + 2(0)) / 4 = 1/4     | 3/8 - 1/4 = 0.125    |
|             | **propDelayed > 0.21**          | **1 - ((0/1)^2 + (1/1)^2) = 0**     | **1 - ((3/3)^2 + (0/3)^2) = 0**     | **(1(0) + 3(0)) / 4 = 0**         | **3/8 - 0 = 0.375**      |
|             | meanDelayAcrossFlights > 13 | 1 - ((1/2)^2 + (1/2)^2) = 2/4   | 1 - ((2/2)^2 + (0/2)^2) = 0     | (2(2/4) + 2(0)) / 4 = 1/4     | 3/8 - 1/4 = 0.125    |

After the 3rd split, we have classified the training set perfectly and so no more splits can be made. Although this represents one potential stop criterion, with larger datasets perfectly classifying the data is highly unlikely so other stop criterion are used, such as a hyperparameter for maximum depth of the tree. For this toy example, training on the data set results in the following tree:

<img src="https://github.com/KarenKailun/img/raw/main/toytree2.png", width=500>

Now that we’ve shown how to train a single decision tree, building up to the random forests algorithm is relatively straightforward. We start by setting a hyperparameter \\(n\\) for how many total decision trees to create. Then we can apply bootstrap aggregating (bagging), which involves training each of the \\(n\\) trees on different resampled (with replacement) sets of the training data. With \\(n\\) different decision trees, we can perform inference by inputting test data into each tree and using a majority vote method to give a prediction for the class of each test input. For example, for the decision tree displayed above, if we wanted to predict a test data point with the following feature values: (DEP_DELAY_NEW_p = 9, propDelayed = 0.12, meanDelayAcrossFlights = 5.6), we would predict a class of 1 (delayed flight) because this data point falls into the right branch of the 2nd split point. In a random forests algorithm, this same data point would be inputted into each of \\(n\\) trees, so we would also end up with \\(n\\) number of predictions; a majority vote method would then take whatever class was predicted by the most trees and that class would be the overall prediction of the random forest for this particular test data point.

We chose to use random forests as our “main” algorithm for this project because it provides several advantages: (1) bagging helps combat overfitting to training data by reducing variance in model predictions, (2) categorical variables can be handled relatively easily without the need for potentially expensive transformations such as one-hot encoding, (3) feature scaling is not necessary since each split point only evaluates a single feature at a time, and (4) Gini gain can be used to measure how well each feature improves classification, thus giving an idea of feature importance (as discussed in Model Results section). On the other hand, a major disadvantage of random forests is that re-training the model requires using all the data again (as opposed to other algorithms that may be able to simply update existing parameters by taking in newly available data). Therefore, this model should ideally be trained offline and at relatively longer time intervals after a significant amount of new data has been collected.

### 5.2 Full random forest implementation

The implementation of the full random forest model required indexing the categorical variables (OP carrier, origin, destination, origin of previous flight, and day of week), though one-hot encoding and scaling were deferred (see `3.3, Importance of algorithm choice on feature transformation choices`). 

The pipeline for training, scoring, and validation was as follows:

<img src="https://github.com/KarenKailun/img/raw/main/trainvalpipe.png", width=800>

In [0]:
# RANDOM FOREST TRAIN-EVAL PIPELINE

def train_eval(traindata=train_mini, testdata = test_fin, features="features", trees=20, depth=5, weights=True):

  # Train
  start_time = time.time()
  if weights:
    m = RandomForestClassifier(labelCol="DEP_DEL15", featuresCol=features, maxBins = 369, numTrees=trees, maxDepth = depth, maxMemoryInMB=1000, weightCol="classWeights")
  else:
    m = RandomForestClassifier(labelCol="DEP_DEL15", featuresCol=features, maxBins = 369, numTrees=trees, maxDepth = depth, maxMemoryInMB=1000)
  m_model=m.fit(traindata)
  traintime = time.time() - start_time

  # Eval
  start_time = time.time()
  train_pred = m_model.transform(traindata)
  test_pred = m_model.transform(testdata)
  predicttime = time.time() - start_time
  
  test_preds = [prediction[0] for prediction in test_pred.select('prediction').collect()]
  test_labels = [label[0] for label in test_pred.select('DEP_DEL15').collect()]
  
  train_preds = [prediction[0] for prediction in train_pred.select('prediction').collect()]
  train_labels = [label[0] for label in train_pred.select('DEP_DEL15').collect()]
  
  cm = confusion_matrix(test_labels, test_preds)
  
  acc = (cm[0,0] + cm[1,1])/np.sum(cm)
  prec = cm[1,1]/(cm[1,1] + cm[0,1])
  recall = cm[1,1]/(cm[1,1] + cm[1,0])
  f1 = 2*(prec*recall)/(prec+recall)
  
  cm2 = confusion_matrix(train_labels, train_preds)
  prec2 = cm2[1,1]/(cm2[1,1] + cm2[0,1])
  recall2 = cm2[1,1]/(cm2[1,1] + cm2[1,0])
  trainf1 = 2*(prec2*recall2)/(prec2+recall2)
  
  # examine feature importance
  importance = DenseVector(m_model.featureImportances)
  rf_importance = pd.DataFrame(list(zip(feature_names, importance)), columns=["feature", "importance"]).sort_values(by="importance", ascending=False)
  
  params = [features, trees, depth, traintime, predicttime]
  metrics = [cm, acc, prec, recall, f1, cm2, trainf1]
  
  print("params", params)
  print("metrics", metrics)
  
  return params, metrics, test_pred, rf_importance

**Hyperparameters** were tuned in a very limited way using a small random sample of the data (0.1%) to allow quick iteration over the parameter space. The hyperparameters were tuned for maximum tree depth and number of trees. Because we used a small sample of the data, the hyperparameter tuning is illustrative only. F1 was used as the metric for tuning. The outcome class was weighted according to frequency. 

The results of limited hyperparameter tuning are shown below:

<img src="https://github.com/KarenKailun/img/raw/main/tuning.png", width=800>

It is not practical to apply the results of limited hyperparameter searching for a small sample of the data --- this was intended simply to illustrate the process of tuning. We selected a max depth of 5 and 20 trees to train with the full dataset. The most important features are shown below:

<img src="https://github.com/KarenKailun/img/raw/main/feat_imp_color.png", width=800>

#### *Targeted model experimentation*

We performed targeted experiments to evaluate random forest model performance in response to the following changes:
* Initial: Full data, full features, 20 trees, 5 max depth
* Reduced features: Features were reduced according to a threshold importance of >0.001 from the baseline model
* Previous year data: Only 1 prior year of data was used for training
* 20 → 400 trees: Large increase in number of trees
* Majority class undersampling: Undersampled majority class rather than using class weighting

Note that training and prediction times may not be comparable across experiments due to variable cluster resources. Also, the F1, precision, and recall are reported for the delay class, rather than overall metrics averaged over both classes. These metrics are also not weighted according to class balance. 

|                 | **Initial**: full data, full features, 20 trees, 5 depth | **Reduced features**, full data, 20 trees, 5 depth | **Previous year data**, full features, 20 trees, 5 depth | **400 trees**, previous year data, full features, 5 depth | **Majority undersampled**, previous year data, full features, 100 trees, 5 depth |
|-----------------|------------------------------------------------------|------------------------------------------------|------------------------------------------------------|-------------------------------------------------------|------------------------------------------------------------------------------|
| $$Accuracy$$       | 0.8526                                               | 0.8521                                         | **0.8530**                                               | 0.8515                                                | 0.8504                                                                       |
| $$F1_{delay}$$              | 0.5556                                               | 0.5549                                         | **0.5565**                                               | 0.5547                                                | 0.5536                                                                       |
| $$Precision_{delay}$$       | 0.5997                                               | 0.5978                                         | **0.6012**                                               | 0.5953                                                | 0.5908                                                                       |
| $$Recall_{delay}$$          | 0.5176                                               | 0.5177                                         | 0.5179                                               | 0.5192                                                | **0.5208**                                                                       |
| Training time   | 158 seconds                                          | 148 seconds                                    | **127 seconds**                                          | 600 seconds                                           | 146 seconds                                                                  |
| Prediction time | **0.14 seconds**                                         | 0.16 seconds                                   | 0.15 seconds                                         | 0.56 seconds                                          | 0.17 seconds                                                                 |

In [0]:
# RANDOM UNDERSAMPLING OF MAJORITY CLASS

majordf = train_18.filter(f.col("DEP_DEL15")==0)

sampled_majority_df = majordf.sample(False, .2105322)

minordf = train_18.filter(f.col("DEP_DEL15")==1)

train_18_under = sampled_majority_df.unionAll(minordf)

In [0]:
# SMOTE-NC (NOT USED)

from imblearn.over_sampling import SMOTENC
sm = SMOTENC(random_state=24601, categorical_features=[7, 10], sampling_strategy='minority', n_jobs=3)
X_res, Y_res = sm.fit_resample(X, Y)

## 6. Conclusions

For the random forest model, **the most important features were ones that we had engineered**, which highlighted the importance of thoughtful feature selection and engineering. We focused on precision and F1 in evaluating our models, assuming it is most important to airlines and to passengers for the predictions to have high precision. 

We did not perform full hyperparameter tuning, opting for targeted experiments instead. **Reducing the number of features** caused slightly lower precision, but the savings in training time were only 10 seconds compared with using all of the features originally selected. This suggests that initial feature selection process was adequate, and that there is little benefit to reducing the feature space further. 

**Restricting the training data to only the previous year** led to better accuracy, precision, and recall, and it reduced training time by 32 seconds. This suggests that older training data may be adding more noise, as airlines adapt their procedures over time and may improve their ability to recover from delays.

We tried mitigating class imbalance by 1) using class weights in the model training and 2) **random undersampling of the majority class**. While undersampling the majority class led to better training F1, it did not improve performance on the test set. Recall was slightly higher in the undersampled data; this may be desirable for passengers who may want a lower false negatives in predictions. We implemented SMOTE-NC from the `imblearn` module, but found that SMOTE did not improve results for the random forest model and was computationally expensive. Our findings with SMOTE are supported <a href="https://journalofbigdata.springeropen.com/articles/10.1186/s40537-019-0274-4">in the literature</a>. 

The confusion matrix for our best performing model, which used only the previous year of training data, shows relatively low precision and lower recall for the delay class (1).

<img src="https://github.com/KarenKailun/img/raw/main/cm.png", width=500>

However, viewing the predicted probabilities gives more insights. A higher probability threshold for predicting delay would greatly improve precision, and there is a natural threshold around 0.8 based on probabilities outputted by the random forest model (shown in figure below). **Using the probability, rather than a strict prediction threshold, would allow airlines and customers to adjust their actions according to a delay probability**, similar to a rain forecast. 

Rather than interpreting delay probability < 0.6 as "no delay", users could interpret this as "not enough information to predict delay at this time." For delay probabilities of 0.6-0.8, users can be warned of a reasonable chance of delay; for probabilities >0.8, users can plan on a likely delay.

<img src="https://github.com/KarenKailun/img/raw/main/prob_class.png", width=600>

## 7. Application of course concepts

**Scalability**

While good results have been reported from the literature using smaller datasets, we aimed to preserve as much useful data as possible for our model training. 
* *Filtering the data early and often* was key to being able to scale the data processing. The strongest example of this was our handling of weather data. Weather data were filtered to only those stations at airports (688 unique airport stations out of 15,159 unique stations), and observations were limited to only those with valid temperature readings to improve the quality of the retained data. While we could have added additional nearby weather stations or additional hours of data, our EDA and literature review suggested that weather was not one of the more important factors in predicting delay. We deferred additional processing of weather data in favor of more impactful feature engineering, such as delay in the previous leg of the flight. 
* Our *joining strategies* also emphasized scalable practices. We favored equi joins over non-equi joins, opting for simplifications such as flooring the flight data to the hour and performing exact joins with hourly averages for weather readings. These strategies, while not as exact or as flexible as calculating time differences between flight departures and weather readings, allowed data to be joined quickly. Smaller joins, such as the small table of airport weather stations joining to the large weather data, were broadcast joins. In addition to filtering data, we also aggregated weather data to the hour and airport to simplify the flight data to a many-to-one join rather than a many-to-many join. 
* *Minimizing sorting* was important for reducing redundant processing. With time dependencies for linking legs of a flight in order, sorting is inevitable. We ensured that we sorted on the minimum necessary data and features and cached the data for additional processing. For the train-test split, rather than sorting the entire dataset in time order, we simplified by filtering out a test dataset based on year. 
* Our choice of the random forest *algorithm* allowed parallelization of the training process. 
* We were attentive to *memory and disk management*. We cached data to avoid re-running computationally intensive code and were also mindful of unpersisting data when no longer needed in memory. We wrote data intermittently to DBFS, which also allowed more resilience to cluster outages. This allows better resilience in a production environment. 

**Feature selection and transformation**

Creating our feature matrix also highlighted important concepts:
* EDA suggested few obviously relevant features. In the *bias-variance trade-off*, we wanted sufficient *model complexity* to capture the non-obvious interactions between features that might be relevant for prediction, while also not overfitting the data and expanding the feature space with irrelevant features. Iterative *feature selection* helps to reduce the training time and model complexity, though we chose not to drop features from the final model because doing so did not appreciably change the performance or training time of our model. 
* We created a simple pipeline for indexing the categorical data and assembling the feature vectors. However, certain transformations, such as normalization, one-hot encoding, and Fourier transformation of cyclical features, were either not needed or were detrimental to a random forest algorithm. 

**Model assumptions**

* For our baseline logistic regression model, we had to assume a linear relationship between the predictors and the logit of the outcome. A highly complex feature space can challenge this assumption. Logistic regression assumes independence of all observations; in our data, flights that were part of the same multi-leg trip are arguably not independent, but we explicitly modeled their interaction by including data about the previous leg of the flight. Logistic regression also assumes no collinearity between variables; we preemptively dropped several variables that were expected to be collinear, such as month and quarter or air and dewpoint temperatures. 
* We chose a random forest model for our main algorithm. Random forest is a non-parametric method that has the advantage of making relatively few assumptions about the data. However, a limitation of random forest models is that they are prone to overfitting if the hyperparameter for number of decision trees is not large enough. This may be why our performance with the test set was unimpressive. Increasing the number of trees should decrease overfitting and make the model more generalizable (*bias-variance tradeoff*).

### References
* Loris Belcastro, Fabrizio Marozzo, Domenico Talia, and Paolo Trunfio. 2016. Using Scalable Data Mining for Predicting Flight Delays. ACM Trans. Intell. Syst. Technol. 8, 1, Article 5 (October 2016), 20 pages. DOI:https://doi.org/10.1145/2888402
* Chakrabarty. Navoneel, "A Data Mining Approach to Flight Arrival Delay Prediction for American Airlines", 2019 9th Annual Information Technology Electromechanical Engineering and Microelectronics Conference (IEMECON). IEEE, 2019.
* Patgiri, R., Hussain, S., & Nongmeikapam, A. (2020). Empirical Study on Airline Delay Analysis and Prediction. ArXiv, abs/2002.10254.
* Etani, N. (2019). Development of a predictive model for on-time arrival flight of airliner by discovering correlation between flight and weather data. Journal of Big Data, 6, 1-17.
* Hasanin, T., Khoshgoftaar, T.M., Leevy, J.L. et al. Severely imbalanced Big Data challenges: investigating data sampling approaches. J Big Data 6, 107 (2019). https://doi.org/10.1186/s40537-019-0274-4

### Data sources and metadata
* Bureau of Transportation Statistics website: https://www.transtats.bts.gov/
* Cost of flight delays:
https://www.faa.gov/data_research/aviation_data_statistics/media/cost_delay_estimates.pdf
* Flight data: http://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236&DB_Short_Name=On-Time
* Data dictionary for flight data: https://www.transtats.bts.gov/Glossary.asp?index=C
* Weather data: National Oceanic and Atmospheric Administration repository 
* Weather code descriptions: https://www.ncei.noaa.gov/data/global-hourly/doc/isd-format-document.pdf
* Airport Location: https://www.aviationweather.gov/docs/metar/stations.txt